## Ingest images

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from buck.analysis.basics import ingest_images

fpath = "..\\images\\squared\\*.png"
images,ages = ingest_images(fpath)

KeyboardInterrupt: 

## Split datasets

In [ ]:
from buck.analysis.basics import split_data

Xtr_og, ytr_og, Xval, yval, Xte, yte_onehot, ages, l_map = split_data(images, ages)

## Homogenize data across classes

In [ ]:
from buck.analysis.basics import homogenize_data

augment_multiplier = 25
X_train_pca, y_train_flat, X_test_pca, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, augment_multiplier)

## Comparison classifier models

In [ ]:
from buck.analysis.optimize_model import compare_models

best_model = compare_models(X_train_pca, y_train_flat, X_test_pca, y_true, num_classes, label_mapping)

## General cleanup before optimization

In [ ]:
import gc

# Force garbage collection
gc.collect()

# You can also check how many objects were collected
collected = gc.collect()
print(f"Garbage collector: collected {collected} objects.")

## Optimize top-performing classifier

In [ ]:
from buck.classifiers.gradient_boost import _optimize_gradient_boost

optimals, accuracy = _optimize_gradient_boost(X_train_pca, y_train_flat, X_test_pca, y_true, cycles=2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure()
plt.plot(accuracy)
title = f"Max. accuracy: {np.max(accuracy)*100:.2f}%"
plt.title(title)
plt.show()